In [ ]:
import unsloth

In [ ]:
from unsloth import FastVisionModel # FastLanguageModel for LLMs
import torch

In [ ]:
model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Qwen2.5-VL-7B-Instruct-bnb-4bit",
    load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing=True, # True or "unsloth" for long context
)

In [ ]:
import PIL
from PIL import Image
import numpy as np
image = None
with Image.open('../test_images/page-01.png') as im:
    image = np.array(im)
image

In [ ]:
FastVisionModel.for_inference(model) # Enable for inference!

instruction = "Please convert the image to HTML format."

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 20000,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("unsloth/Radiology_mini", split="train")

In [ ]:
print(dataset[0]["image"])